In [70]:
from pycaret.classification import *
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json
import requests
from bs4 import BeautifulSoup
import statistics
import math

In [21]:
open_file = open('model_columns.pkl', "rb")
model_columns = list(pickle.load(open_file))
open_file.close()

In [22]:
scraped_cols = ['Total shots','Expected goals (xG)','Offsides','Accurate passes percentage','score','Corners','Fouls committed','Shots on target','Blocks']
pred_cols = dict(zip(model_columns, scraped_cols))

In [23]:
model = load_model('lr_model')

Transformation Pipeline and Model Successfully Loaded


In [243]:
today = f"{datetime.now():%Y%m%d}"
game_data = fotmob_game_data(f'https://www.fotmob.com/?date={today}')
all_odds = get_all_odds_links()
days_data = connect_links(game_data, all_odds)

In [244]:
days_data

{'2022-08-27 08:23:00': [['southampton-vs-manchester-united',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900970/matchfacts/southampton-vs-manchester-united',
   'https://www.oddschecker.com/football/english/premier-league/southampton-v-man-utd/winner'],
  ['sunderland-vs-norwich-city',
   'ENGLAND - CHAMPIONSHIP',
   'https://www.fotmob.com/match/3915353/matchfacts/sunderland-vs-norwich-city',
   'https://www.oddschecker.com/football/english/championship/sunderland-v-norwich/winner']],
 '2022-08-27 10:23:00': [['hertha-bsc-vs-borussia-dortmund',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903577/matchfacts/hertha-bsc-vs-borussia-dortmund',
   'https://www.oddschecker.com/football/germany/bundesliga/hertha-berlin-v-borussia-dortmund/winner'],
  ['tsg-hoffenheim-vs-fc-augsburg',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903576/matchfacts/tsg-hoffenheim-vs-fc-augsburg',
   'https://www.oddschecker.com/football/germany/bundesliga/ts

In [39]:
days_data = {
 '2022-08-27 10:53:00': [['brentford-vs-everton',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900964/matchfacts/brentford-vs-everton',
   'https://www.oddschecker.com/football/english/premier-league/aston-villa-v-west-ham/winner'],
  ['brighton-&-hove-albion-vs-leeds-united',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900965/matchfacts/brighton-&-hove-albion-vs-leeds-united',
   'https://www.oddschecker.com/football/english/premier-league/aston-villa-v-west-ham/winner'],
  ['chelsea-vs-leicester-city',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900966/matchfacts/chelsea-vs-leicester-city',
   'https://www.oddschecker.com/football/english/premier-league/aston-villa-v-west-ham/winner'],
  ['liverpool-vs-afc-bournemouth',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900967/matchfacts/liverpool-vs-afc-bournemouth',
   'https://www.oddschecker.com/football/english/premier-league/aston-villa-v-west-ham/winner'],
  ['manchester-city-vs-crystal-palace',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900968/matchfacts/manchester-city-vs-crystal-palace',
   'https://www.oddschecker.com/football/english/premier-league/aston-villa-v-west-ham/winner']]}

In [17]:
days_data = {
 '2022-08-27 11:23:00': [['go-ahead-eagles-vs-sparta-rotterdam',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900369/matchfacts/go-ahead-eagles-vs-sparta-rotterdam',
   'https://www.oddschecker.com/football/netherlands/eredivisie/go-ahead-eagles-v-sparta-rotterdam/winner'],
  ['maritimo-vs-portimonense',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937395/matchfacts/maritimo-vs-portimonense',
   'https://www.oddschecker.com/football/portugal/primeira-liga/maritimo-v-portimonense/winner']],
 '2022-08-27 11:53:00': [['auxerre-vs-strasbourg',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904420/matchfacts/auxerre-vs-strasbourg',
   'https://www.oddschecker.com/football/france/ligue-1/auxerre-v-strasbourg/winner'],
  ['austria-lustenau-vs-salzburg',
   'AUSTRIA - BUNDESLIGA',
   'https://www.fotmob.com/match/3917292/matchfacts/austria-lustenau-vs-salzburg',
   'https://www.oddschecker.com/football/austria/bundesliga/austria-lustenau-v-fc-salzburg/winner'],
  ['hartberg-vs-ried',
   'AUSTRIA - BUNDESLIGA',
   'https://www.fotmob.com/match/3917293/matchfacts/hartberg-vs-ried',
   'https://www.oddschecker.com/football/austria/bundesliga/hartberg-v-sv-ried/winner']],
 '2022-08-27 12:23:00': [['elche-vs-real-sociedad',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917950/matchfacts/elche-vs-real-sociedad',
   'https://www.oddschecker.com/football/spain/la-liga-primera/elche-v-real-sociedad/winner']],
 '2022-08-27 12:53:00': [['luzern-vs-sion',
   'SWITZERLAND - SUPER LEAGUE',
   'https://www.fotmob.com/match/3905448/matchfacts/luzern-vs-sion',
   'https://www.oddschecker.com/football/switzerland/super-league/lucerne-v-fc-sion/winner']],
 '2022-08-27 13:08:00': [['cercle-brugge-vs-zulte-waregem',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3916963/matchfacts/cercle-brugge-vs-zulte-waregem',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/cercle-brugge-v-zulte-waregem/winner'],
  ['oh-leuven-vs-oostende',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3916962/matchfacts/oh-leuven-vs-oostende',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/oh-leuven-v-kv-oostende/winner']],
 '2022-08-27 13:23:00': [['arsenal-vs-fulham',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900962/matchfacts/arsenal-vs-fulham',
   'https://www.oddschecker.com/football/english/premier-league/arsenal-v-fulham/winner'],
  ['cremonese-vs-torino',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919099/matchfacts/cremonese-vs-torino',
   'https://www.oddschecker.com/football/italy/serie-a/cremonese-v-torino/winner'],
  ['juventus-vs-roma',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919102/matchfacts/juventus-vs-roma',
   'https://www.oddschecker.com/football/italy/serie-a/juventus-v-roma/winner'],
  ['bayern-m%C3%BCnchen-vs-borussia-mgladbach',
   'GERMANY - BUNDESLIGA',
   'https://www.fotmob.com/match/3903571/matchfacts/bayern-m%C3%BCnchen-vs-borussia-mgladbach',
   'https://www.oddschecker.com/football/germany/bundesliga/bayern-munich-v-borussia-mgladbach/winner']],
 '2022-08-27 13:38:00': [['vitesse-vs-rkc-waalwijk',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900370/matchfacts/vitesse-vs-rkc-waalwijk',
   'https://www.oddschecker.com/football/netherlands/eredivisie/vitesse-v-rkc/winner']],
 '2022-08-27 13:53:00': [['boavista-vs-benfica',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937385/matchfacts/boavista-vs-benfica',
   'https://www.oddschecker.com/football/portugal/primeira-liga/boavista-v-benfica/winner']],
 '2022-08-27 14:23:00': [['rayo-vallecano-vs-mallorca',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917957/matchfacts/rayo-vallecano-vs-mallorca',
   'https://www.oddschecker.com/football/spain/la-liga-primera/rayo-vallecano-v-mallorca/winner'],
  ['lask-vs-altach',
   'AUSTRIA - BUNDESLIGA',
   'https://www.fotmob.com/match/3917294/matchfacts/lask-vs-altach',
   'https://www.oddschecker.com/football/austria/bundesliga/lask-linz-v-scr-altach/winner']],
 '2022-08-27 14:53:00': [['feyenoord-vs-fc-emmen',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900371/matchfacts/feyenoord-vs-fc-emmen',
   'https://www.oddschecker.com/football/netherlands/eredivisie/feyenoord-v-fc-emmen/winner']],
 '2022-08-27 15:23:00': [['servette-vs-grasshopper',
   'SWITZERLAND - SUPER LEAGUE',
   'https://www.fotmob.com/match/3905447/matchfacts/servette-vs-grasshopper',
   'https://www.oddschecker.com/football/switzerland/super-league/servette-v-grasshoppers/winner']],
 '2022-08-27 15:38:00': [['milan-vs-bologna',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919105/matchfacts/milan-vs-bologna',
   'https://www.oddschecker.com/football/italy/serie-a/ac-milan-v-bologna/winner'],
  ['spezia-vs-sassuolo',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919108/matchfacts/spezia-vs-sassuolo',
   'https://www.oddschecker.com/football/italy/serie-a/spezia-v-sassuolo/winner'],
  ['st.truiden-vs-kv-mechelen',
   'BELGIUM - FIRST DIVISION A',
   'https://www.fotmob.com/match/3916964/matchfacts/st.truiden-vs-kv-mechelen',
   'https://www.oddschecker.com/football/belgium/jupiler-pro-league/sint-truidense-v-kv-mechelen/winner']],
 '2022-08-27 15:53:00': [['lens-vs-rennes',
   'FRANCE - LIGUE 1',
   'https://www.fotmob.com/match/3904425/matchfacts/lens-vs-rennes',
   'https://www.oddschecker.com/football/france/ligue-1/lens-v-rennes/winner'],
  ['sc-heerenveen-vs-fortuna-sittard',
   'NETHERLANDS - EREDIVISIE',
   'https://www.fotmob.com/match/3900372/matchfacts/sc-heerenveen-vs-fortuna-sittard',
   'https://www.oddschecker.com/football/netherlands/eredivisie/heerenveen-v-fortuna-sittard/winner']],
 '2022-08-27 16:23:00': [['portland-timbers-vs-seattle-sounders-fc',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787582/matchfacts/portland-timbers-vs-seattle-sounders-fc',
   'https://www.oddschecker.com/football/world/usa/mls/portland-timbers-v-seattle-sounders-fc/winner'],
  ['sporting-cp-vs-chaves',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937387/matchfacts/sporting-cp-vs-chaves',
   'https://www.oddschecker.com/football/portugal/primeira-liga/sporting-v-chaves/winner']],
 '2022-08-27 16:53:00': [['almeria-vs-sevilla',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917953/matchfacts/almeria-vs-sevilla',
   'https://www.oddschecker.com/football/spain/la-liga-primera/almeria-v-sevilla/winner']],
 '2022-08-27 18:53:00': [['cruz-azul-vs-queretaro',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888552/matchfacts/cruz-azul-vs-queretaro',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/cruz-azul-v-queretaro/winner']],
 '2022-08-27 19:53:00': [['minnesota-united-vs-houston-dynamo-fc',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787583/matchfacts/minnesota-united-vs-houston-dynamo-fc',
   'https://www.oddschecker.com/football/world/usa/mls/minnesota-united-v-houston-dynamo/winner'],
  ['charlotte-fc-vs-toronto-fc',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787728/matchfacts/charlotte-fc-vs-toronto-fc',
   'https://www.oddschecker.com/football/world/usa/mls/charlotte-fc-v-toronto-fc/winner']],
 '2022-08-27 20:23:00': [['new-york-red-bulls-vs-inter-miami-cf',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787584/matchfacts/new-york-red-bulls-vs-inter-miami-cf',
   'https://www.oddschecker.com/football/world/usa/mls/new-york-red-bulls-v-inter-miami-cf/winner'],
  ['fc-cincinnati-vs-columbus-crew',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787605/matchfacts/fc-cincinnati-vs-columbus-crew',
   'https://www.oddschecker.com/football/world/usa/mls/fc-cincinnati-v-columbus-crew/winner']],
 '2022-08-27 20:53:00': [['philadelphia-union-vs-colorado-rapids',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787606/matchfacts/philadelphia-union-vs-colorado-rapids',
   'https://www.oddschecker.com/football/world/usa/mls/philadelphia-union-v-colorado-rapids/winner']],
 '2022-08-27 20:58:00': [['tigres-vs-necaxa',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888553/matchfacts/tigres-vs-necaxa',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/tigres-v-club-necaxa/winner']],
 '2022-08-27 21:23:00': [['chicago-fire-fc-vs-cf-montreal',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787727/matchfacts/chicago-fire-fc-vs-cf-montreal',
   'https://www.oddschecker.com/football/world/usa/mls/chicago-fire-v-cf-montreal/winner']],
 '2022-08-27 21:53:00': [['sporting-kansas-city-vs-san-jose-earthquakes',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787607/matchfacts/sporting-kansas-city-vs-san-jose-earthquakes',
   'https://www.oddschecker.com/football/world/usa/mls/sporting-kansas-city-v-san-jose-earthquakes/winner']],
 '2022-08-27 22:53:00': [['fc-dallas-vs-real-salt-lake',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787608/matchfacts/fc-dallas-vs-real-salt-lake',
   'https://www.oddschecker.com/football/world/usa/mls/fc-dallas-v-real-salt-lake/winner']],
 '2022-08-27 22:58:00': [['guadalajara-vs-club-universidad-nacional',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888554/matchfacts/guadalajara-vs-club-universidad-nacional',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/tigres-v-club-necaxa/winner']]}

In [40]:
c = days_data.copy()
todays_data = []

while True:
    for clock in c.copy().keys():
        if datetime.strptime(clock, "%Y-%m-%d %H:%M:%S") <= datetime.now():
            stats_urls = [x[2] for x in c[clock]]
            odds_urls = [x[3] for x in c[clock]]
            print(stats_urls, odds_urls)
            with concurrent.futures.ThreadPoolExecutor() as executor:
                r = executor.map(stats_odds, stats_urls, odds_urls)
            for result in r:
                todays_data.append(result)
            del c[clock]
    print(f'{len(c)} halftimes left -- {len(todays_data)}\n')
    if len(c) == 0:
        print('all done')
        break
    time.sleep(60)

open_file = open(f'{today}_data.pkl', "wb")
pickle.dump(todays_data, open_file)
open_file.close()

['https://www.fotmob.com/match/3900964/matchfacts/brentford-vs-everton', 'https://www.fotmob.com/match/3900965/matchfacts/brighton-&-hove-albion-vs-leeds-united', 'https://www.fotmob.com/match/3900966/matchfacts/chelsea-vs-leicester-city', 'https://www.fotmob.com/match/3900967/matchfacts/liverpool-vs-afc-bournemouth', 'https://www.fotmob.com/match/3900968/matchfacts/manchester-city-vs-crystal-palace'] ['https://www.oddschecker.com/football/english/premier-league/aston-villa-v-west-ham/winner', 'https://www.oddschecker.com/football/english/premier-league/aston-villa-v-west-ham/winner', 'https://www.oddschecker.com/football/english/premier-league/aston-villa-v-west-ham/winner', 'https://www.oddschecker.com/football/english/premier-league/aston-villa-v-west-ham/winner', 'https://www.oddschecker.com/football/english/premier-league/aston-villa-v-west-ham/winner']
not halftime just yet ... sleeping

not halftime just yet ... sleeping

not halftime just yet ... sleeping

not halftime just yet

In [35]:
todays_data

[{'Expected goals (xG)': ['2.50', '1.12'],
  'Total shots': ['19', '14'],
  'Big chances': ['3', '3'],
  'Big chances missed': ['2', '2'],
  'Accurate passes': ['424', '296'],
  'Accurate passes percentage': [0.83, 0.79],
  'Fouls committed': ['8', '8'],
  'Offsides': ['0', '2'],
  'Corners': ['8', '5'],
  'Shots off target': ['9', '2'],
  'Shots on target': ['5', '7'],
  'Blocked shots': ['5', '5'],
  'Hit woodwork': ['3', '0'],
  'Shots inside box': ['13', '8'],
  'Shots outside box': ['6', '6'],
  'xG first half': ['0.92', '0.67'],
  'xG second half': ['1.58', '0.44'],
  'xG open play': ['0.91', '0.83'],
  'xG set play': ['1.59', '0.28'],
  'xG on target (xGOT)': ['1.58', '1.50'],
  'Passes': ['508', '377'],
  'Own half': ['179', '150'],
  'Opposition half': ['245', '146'],
  'Accurate long balls': ['17', '26'],
  'Accurate long balls percentage': [0.36, 0.38],
  'Accurate crosses': ['6', '3'],
  'Accurate crosses percentage': [0.2, 0.23],
  'Throws': ['25', '12'],
  'Yellow cards':

In [15]:
def predicted_probs(results):
    d = {}
    for k, v in results.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    for_pred = [[d[key] for key in pred_cols.values()]]
    pred = model[-1].predict_proba(for_pred)
    return pred

In [14]:
def make_bet(decimal_odds, predicted_prob, k_weight):
    if 1/decimal_odds >= predicted_prob:
        return 'no value'
    else:
        return kelly(predicted_prob, decimal_odds, k_weight)

In [13]:
def print_advice(results):
    try:
        d_h_a = ['draw'] + results['odds_url'].split('/')[-2].split('-v-')
        order_decimal_odds = [getname(odds_name, list(results['odds'].keys())) for odds_name in d_h_a]
        print(order_decimal_odds)
        p_probs = predicted_probs(results)[0]
        predicted = np.argmax(p_probs)
        decimal_odds = [results['odds'][key] for key in order_decimal_odds]
        print(decimal_odds)
        bet_made = {}
        print('\033[1m' + results['match_name'] + '\033[0m')
        print(f'Predicted winner is {predicted}\n')
        for p_prob, d_odd, win in zip(p_probs, decimal_odds, ['draw', 'home','away']):
            p_prob = round(p_prob, 3)
            d_odd = round(d_odd, 3)
            size = make_bet(d_odd, p_prob, 0.5)
            print(f'{win}: predicted prob: {p_prob*100}% - oddschecker: {round(1/d_odd*100,2)}% ({d_odd}) ---- bet {size}')
            bet_made[win] = size
        print('\n')
        bet_made['odds_checker_sum'] = np.sum([1/o for o in decimal_odds])
        bet_made['winner'] = predicted
        bet_made['predicted_probs'] = np.round(p_probs, 3)
        bets = {'bet':bet_made} 
        row = {**results, **bets}
    except:
        row = results
    return row

In [4]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [3]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [32]:
def get_stats(url):
    driver = webdriver.Chrome()
    try:
        driver.implicitly_wait(1)
        driver.get(url)
        ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        if ht != 'Half-Time':
            print('not halftime just yet ... sleeping\n')
#             time.sleep(3*60)
            ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
#         button.click()
        div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
        t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
              if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
        t['score'] = [score[0], score[2]]
        posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
        t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
        game_stats = {}
        for name, stats in t.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
#         game_stats = {k: float(v[0]) - float(v[1]) for k, v in game_stats.items()}
        match_name = url.split('/')[-1]
        game_stats['match_name'] = match_name
        game_stats['half_time'] = ht
        game_stats['url'] = url
    except:
        game_stats = {}
    driver.quit()
    return game_stats

In [12]:
xg_leagues = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership',
             'ITALY - SERIE A':'italy/serie-a'}

def fotmob_game_data(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)
    driver.get(url)
    divs = driver.find_elements(By.XPATH, '//div[@class="css-aoqfcq-Group e7pc1841"]')
    today = url[-8:]
    league = {}
    for count, bucket in enumerate(divs):
        league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
        if league_name not in xg_leagues.keys():
            continue
        url = bucket.find_elements(By.TAG_NAME, 'a')
        urls = [u.get_attribute('href') for u in url]
        time = bucket.text
        time = [datetime.strptime(i + today, '%H:%M%Y%m%d') + timedelta(minutes=53) for i in time.split('\n') if ':' in i]
        urls = urls[1:]
        names = [u.split('/')[-1] for u in urls]
        league[league_name] = {n: [u,t] for u, t, n in zip(urls, time, names)}
    return league

In [184]:
db = fotmob_game_data(f'https://www.fotmob.com/?date=20220830')

In [186]:
all_odds = get_all_odds_links()

In [11]:
def get_all_odds_links():
    odds_links = {}
    for f_league, url_league in xg_leagues.items():
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = 'https://www.oddschecker.com/football/' + url_league
        driver.get(url)
        driver.implicitly_wait(3)
        divs = driver.find_elements(By.XPATH, '//a[@class="beta-callout full-height-link whole-row-link"]')
        mt = []
        for div in divs:
            odds_url = div.get_attribute('href')
            mt.append(odds_url)
        odds_links[f_league] = mt
    return odds_links

In [215]:
str(datetime.now())

'2022-08-28 00:42:21.364171'

In [247]:
def connect_links(data, all_odds):
    for l in data.keys():
        fotmob_names = data[l].keys()
        odds_names = {v: k for k, v in zip(all_odds[l],[str(i).split('/')[-2] for i in all_odds[l]])}
        for game in fotmob_names:
            match = getname(game, odds_names.keys())
            try:
                link = odds_names[match]
            except:
                link = ''
            data[l][game] = data[l][game] + [link]
    scraper_data = collections.defaultdict(list)
    for l_name, data in data.items():
        for game_name, l1_t_l2 in data.items():
            scraper_data[str(l1_t_l2[1])].append([game_name, l_name, l1_t_l2[0], l1_t_l2[2]])
    r = {}
    test = scraper_data.copy()
    for date, data in test.copy().items():
        if len(data) > 5:
            x = [data[i:i + 4] for i in range(0, len(data), 4)]
            print((x))
            d = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
            print(d)
            groups = len(x)
            times = [str(d + timedelta(minutes = i)) for i in range(-groups//2+1,groups//2+1, 1)]
            new = dict(zip(times, x))
            del test[date]
            r = {**r, **new}
        else:
            r[date] = data
    scraper_data = dict(sorted(scraper_data.items(), key=lambda x: x[0]))
    return scraper_data

In [248]:
connect_links(db, all_odds)

{'2022-08-30 13:23:00': [['sassuolo-vs-milan',
   'ITALY - SERIE A',
   'https://www.fotmob.com/match/3919117/matchfacts/sassuolo-vs-milan',
   'https://www.oddschecker.com/football/italy/serie-a/sassuolo-v-ac-milan/winner']],
 '2022-08-30 15:23:00': [['crystal-palace-vs-brentford',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900978/matchfacts/crystal-palace-vs-brentford',
   'https://www.oddschecker.com/football/english/premier-league/crystal-palace-v-brentford/winner'],
  ['fulham-vs-brighton-&-hove-albion',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900974/matchfacts/fulham-vs-brighton-&-hove-albion',
   'https://www.oddschecker.com/football/english/premier-league/fulham-v-brighton/winner']],
 '2022-08-30 15:38:00': [['southampton-vs-chelsea',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900979/matchfacts/southampton-vs-chelsea',
   'https://www.oddschecker.com/football/english/premier-league/southampton-v-chelsea/winner

In [52]:
datetime.now() + timedelta(minutes = -2)

datetime.datetime(2022, 8, 27, 14, 52, 10, 22214)

In [201]:
# d = datetime.strptime(datetime.now(), "%Y-%m-%d %H:%M:%S")
x = list(np.zeros(13))
x = [x[i:i + 4] for i in range(0, len(x), 4)]
groups = math.ceil(len(x)/4)
times = [datetime.now() + timedelta(minutes = i) for i in range(-groups//2+1,groups//2+1, 1)]

In [203]:
x

[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0]]

In [146]:
def create(limit, diff):
    length = int(((limit/diff)*2)+1)   
    l = [-limit+i*diff for i in range(length)]
    return l 

In [8]:
def get_odds(url):
    try:
        driver = webdriver.Chrome()
        if url == '':
            return {'odds_url':url, 'odds': '', 'full_odds': ''}
        driver.implicitly_wait(1)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, features="html.parser")
        odds_containers = soup.find_all("div", {"class": "oddsAreaWrapper_o17xb9rs RowLayout_refg9ta"})
        names = soup.find_all("div", {"class": "BetRowLeftBetContent_b2f00kt"})
        names = [n.text for n in names]
        odds_containers = [container.find_all('button') for container in odds_containers]
        odds = [[z.text for z in container] for container in odds_containers]
        odds = [[float(Fraction(j.replace(' ',''))) + 1.0 for j in o][:-3] for o in odds]
        median_odds = [statistics.median(o) for o in odds]
        median_odds = dict(zip(names, median_odds))
        odds = {'odds_url':url, 'odds': median_odds, 'full_odds': odds}
    except:
        return {'odds_url':url, 'odds': '', 'full_odds': ''}
    driver.quit()
    return odds

In [111]:
def stats_odds(stats_url, odds_url):
    stats = get_stats(stats_url)
    time.sleep(20)
    odds = get_odds(odds_url)
    row = {**stats, **odds}
    result = print_advice(row)
    return result